In [1]:
import math
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy.ma as ma
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import cartopy.crs as ccrs
import cartopy.crs as crs
import cartopy.feature as cfeature
import h5py  
from astropy.time import Time
import os
import glob
from osgeo import gdal
import scipy.ndimage
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC
import pprint
import csv

Firstly - Add all ATL08 data and convert to MODIS files.

(Note: may want to download further back then just one year when this is complete)

In [2]:
# Set input directory, and change working directory - plug in D:
inDir = "D:\\masters_data\\ATL08"   # I should change this so I can work from github?
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [3]:
# constants for conversion lat lon -> tile H,V and r,c
R = 6371007.181	    # the radius of the idealizes sphere representing the Earth
T = 1111950	        # the height and width of each MODIS tile in the projection plane
Xmin = -20015109    # the western limit of the projection plane
Ymax = 10007555	    # the northern limit of the projection plane
w = T/2400.0	    # the actual size of a '500m' MODIS sinusoidal grid cell. (would be /1200 for a 1km grid)

In [4]:
# all ATL08 points
all_points = pd.read_csv("D:\\masters_data\\output\\csv\\all_points.csv", header=1)
all_points.columns= ["latitude", "longitude", "mean_canopy", "max_canopy", "min_canopy", "median_canopy", "canopy_openness" ]

In [5]:
latitude = all_points['latitude'][:]
longitude = all_points['longitude'][:]
mean_height = all_points["mean_canopy"][:]
max_height = all_points["max_canopy"][:]
min_height = all_points["min_canopy"][:]
median_canopy = all_points["median_canopy"][:]
canopy_openness = all_points["canopy_openness"][:]

In [6]:
string_latitude = latitude.to_string(index = False)
string_latitude = latitude.replace("\n", ",")

string_longitude = longitude.to_string(index = False)
string_longitude = longitude.replace("\n", ",")

string_mean_height = mean_height.to_string(index = False)
string_mean_height = mean_height.replace("\n", ",")

string_max_height = max_height.to_string(index = False)
string_max_height = max_height.replace("\n", ",")

string_min_height = min_height.to_string(index = False)
string_min_height = min_height.replace("\n", ",")

string_median_canopy = median_canopy.to_string(index = False)
string_median_canopy = median_canopy.replace("\n", ",")

string_canopy_openness = canopy_openness.to_string(index = False)
string_canopy_openness = canopy_openness.replace("\n", ",")

In [7]:
float_latitude = [float(i) for i in string_latitude]
float_longitude = [float(i) for i in string_longitude]
float_mean_height = [float(i) for i in string_mean_height]
float_max_height = [float(i) for i in string_max_height]
float_min_height = [float(i) for i in string_min_height]
float_median_canopy = [float(i) for i in string_median_canopy]
float_canopy_openness = [float(i) for i in string_canopy_openness]

In [8]:
canopy_height_array = np.zeros((2400,2400))
canopy_max_height_array = np.zeros((2400,2400))
canopy_min_height_array = np.zeros((2400,2400))
canopy_median_height_array = np.zeros((2400,2400))
canopy_openness_array = np.zeros((2400,2400))

In [9]:
number = 0
r_list = []
c_list = []
    
for i in latitude:
    lat = float_latitude[number]
    lon = float_longitude[number]
    phi = lat/(180/math.pi) # convert to radians
    lamb = lon/(180/math.pi)
    x = R*lamb*math.cos(phi)
    y = R*phi
    r = int(math.floor(((Ymax-y)%T)/w-0.5))
    c = int(math.floor(((x-Xmin)%T)/w-0.5))
    
    r_list.append(r)
    c_list.append(c)
    number += 1

In [10]:
for ii in range(len(r_list)):
    row = r_list[ii]
    col = c_list[ii]
    canopy_height_array[row, col] = float_mean_height[ii]
    canopy_max_height_array[row, col] = float_max_height[ii]
    canopy_min_height_array[row, col] = float_min_height[ii]
    canopy_median_height_array[row, col] = float_median_canopy[ii]
    canopy_openness_array[row, col] = float_canopy_openness[ii]

In [11]:
H20V08_First_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\first_BA_H20V08.csv", header=None)
H20V08_Second_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\second_BA_H20V08.csv", header=None)

H20V07_First_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\first_BA_H20V07.csv", header=None)
H20V07_Second_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\second_BA_H20V07.csv", header=None)

H19V07_First_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\first_BA_H19V07.csv", header=None)
H19V07_Second_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\second_BA_H19V07.csv", header=None)

H19V08_First_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\first_BA_H19V08.csv", header=None)
H19V08_Second_BA = pd.read_csv("D:\\masters_data\\output\\BA_Categories\\second_BA_H19V08.csv", header=None)

In [46]:
H19V07_Mask = np.zeros((2400, 2400))
# low_low / low_med / low_high

H19V07_Mask[((H19V07_First_BA<1) * (H19V07_Second_BA<1))==1] = 1
H19V07_Mask[((H19V07_First_BA<1) * (H19V07_Second_BA>=1) * (H19V07_Second_BA<7))==1] = 2
H19V07_Mask[((H19V07_First_BA<1) * (H19V07_Second_BA>=7))==1] = 3

# med_low / med_med / med_high
H19V07_Mask[((H19V07_First_BA>=1) * (H19V07_First_BA<7) * (H19V07_Second_BA<1))==1] = 4
H19V07_Mask[((H19V07_First_BA>=1) * (H19V07_First_BA<7) * (H19V07_Second_BA>=1) * (H19V07_Second_BA<7))==1] = 5
H19V07_Mask[((H19V07_First_BA>=1) * (H19V07_First_BA<7) * (H19V07_Second_BA>=7))==1] = 6

#high_low / high_med / high_high
H19V07_Mask[((H19V07_Second_BA<1) * (H19V07_First_BA>=7))==1] = 7
H19V07_Mask[((H19V07_First_BA>=7) * (H19V07_Second_BA>=1) * (H19V07_Second_BA<7))==1] = 8
H19V07_Mask[((H19V07_First_BA>=7) * (H19V07_Second_BA>=7))==1] = 9

# this takes out where canopy height == 0, but means that the colormaps are hardly visible
# H19V07_Mask[(canopy_height_array==0)==1]=0

H19V07_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\first_BA_500-1500_H19V07.csv", header=None)
H19V07_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\second_BA_500-1500_H19V07.csv", header=None)

H19V07_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\first_BA_sub500_H19V07.csv", header=None)
H19V07_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\second_BA_sub500_H19V07.csv", header=None)

H19V07_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\first_BA_above_1500_H19V07.csv", header=None)
H19V07_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\second_BA_above_1500_H19V07.csv", header=None)

# total BA for each rainfall group
H19V07_sub500_total_mask = H19V07_First_BA_sub500 + H19V07_Second_BA_sub500
H19V07_500_1500_total_mask = H19V07_First_BA_500_1500 + H19V07_Second_BA_500_1500
H19V07_above500_total_mask = H19V07_First_BA_above_1500 + H19V07_Second_BA_above_1500

H19V07_total_BA = H19V07_sub500_total_mask + H19V07_500_1500_total_mask + H19V07_above500_total_mask

H19V07_first_BA = H19V07_First_BA_500_1500 + H19V07_First_BA_sub500 + H19V07_First_BA_above_1500
H19V07_second_BA = H19V07_Second_BA_500_1500 + H19V07_Second_BA_sub500 + H19V07_Second_BA_above_1500

# VI
H19V07_data = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H19V07_VI_array.csv", header=None)
H19V07_VI_array_np = H19V07_data.to_numpy()
H19V07_VI_list = H19V07_VI_array_np.flatten().tolist()

H19V07_rain_band_mask = np.zeros((2400, 2400))

for i in range(len(H19V07_sub500_total_mask)):
    for ii in range(len(H19V07_sub500_total_mask)):
        if (H19V07_500_1500_total_mask[i][ii] >= H19V07_sub500_total_mask[i][ii]) & (H19V07_500_1500_total_mask[i][ii] >= H19V07_above500_total_mask[i][ii]):
            H19V07_rain_band_mask[i][ii] = 2
            
        elif (H19V07_above500_total_mask[i][ii] >= H19V07_500_1500_total_mask[i][ii]) & (H19V07_above500_total_mask[i][ii] >= H19V07_sub500_total_mask[i][ii]):
            H19V07_rain_band_mask[i][ii] = 3
            
        elif (H19V07_sub500_total_mask[i][ii] >= H19V07_500_1500_total_mask[i][ii]) & (H19V07_sub500_total_mask[i][ii] >= H19V07_above500_total_mask[i][ii]):
            H19V07_rain_band_mask[i][ii] = 1
            
# add total rainfall
Yearly_H19V07_Rainfall = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H19V07_Rainfall.csv", header=None)

canopy_height_list = canopy_height_array.flatten().tolist()
canopy_max_height_list = canopy_max_height_array.flatten().tolist()
canopy_min_height_list = canopy_min_height_array.flatten().tolist()
canopy_median_height_list = canopy_median_height_array.flatten().tolist()
canopy_openness_list = canopy_openness_array.flatten().tolist()
H19V07_BA_list = H19V07_Mask.flatten().tolist()
H19V07_rain_band_list = H19V07_rain_band_mask.flatten().tolist() 
H19V07_first_BA = H19V07_first_BA.to_numpy()
H19V07_first_BA_list = H19V07_first_BA.flatten().tolist() 
H19V07_second_BA = H19V07_second_BA.to_numpy()
H19V07_second_BA_list = H19V07_second_BA.flatten().tolist() 
H19V07_total_BA = H19V07_total_BA.to_numpy()
H19V07_total_BA_list = H19V07_total_BA.flatten().tolist() 
Yearly_H19V07_Rainfall = Yearly_H19V07_Rainfall.to_numpy()
Yearly_H19V07_Rainfall_list = Yearly_H19V07_Rainfall.flatten().tolist()

H19V07_rain_band_list = map(str, H19V07_rain_band_list)
H19V07_rain_band_list = [i.replace('1.0', 'sub500') for i in H19V07_rain_band_list]
H19V07_rain_band_list = [i.replace('2.0', '500_1500') for i in H19V07_rain_band_list]
H19V07_rain_band_list = [i.replace('3.0', 'above1500') for i in H19V07_rain_band_list]

H19V07_BA_list = map(str, H19V07_BA_list)
H19V07_BA_list = [i.replace('1.0', 'low_low') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('2.0', 'low_med') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('3.0', 'low_high') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('4.0', 'med_low') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('5.0', 'med_med') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('6.0', 'med_high') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('7.0', 'high_low') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('8.0', 'high_med') for i in H19V07_BA_list]
H19V07_BA_list = [i.replace('9.0', 'high_high') for i in H19V07_BA_list]

H19V07_all_data = pd.DataFrame({'mean_canopy': canopy_height_list})

H19V07_all_data['canopy_max_height']= canopy_max_height_list
H19V07_all_data['canopy_min_height']= canopy_min_height_list
H19V07_all_data['canopy_median_height']= canopy_median_height_list
H19V07_all_data['canopy_openness']= canopy_openness_list
H19V07_all_data['BA_category']= H19V07_BA_list
H19V07_all_data['Total rainfall']= Yearly_H19V07_Rainfall_list
H19V07_all_data['First_burns']= H19V07_first_BA_list
H19V07_all_data['Second_burns']= H19V07_second_BA_list
H19V07_all_data['Total burns']= H19V07_total_BA_list
H19V07_all_data['rainfall_band']= H19V07_rain_band_list
H19V07_all_data['Average NDVI']= H19V07_VI_list


H19V07_all_data =H19V07_all_data[H19V07_all_data['mean_canopy'] !=0]

H19V07_all_data['Tile']= "H19V07"

Create H19V07 dataframe

In [18]:
H19V07_all_data

,mean_canopy,canopy_max_height,canopy_min_height,canopy_median_height,canopy_openness,BA_category,First_burns,Second_burns,Total burns,rainfall_band,Average NDVI,Tile
2421,6.04,9.64,2.290,6.42,1.97,low_low,0.0,0.0,0.0,500_1500,979.815534,H19V07
2444,2.26,5.38,0.511,1.74,1.40,low_low,0.0,0.0,0.0,500_1500,986.992718,H19V07
2468,2.53,7.31,0.514,1.18,2.25,low_low,0.0,0.0,0.0,500_1500,972.259709,H19V07
2492,5.72,8.51,0.622,5.94,2.13,low_low,0.0,0.0,0.0,500_1500,976.728155,H19V07
2515,5.20,8.15,0.520,5.08,1.86,low_low,0.0,0.0,0.0,500_1500,978.490291,H19V07
...,...,...,...,...,...,...,...,...,...,...,...,...
5759902,2.32,7.27,0.537,1.46,1.89,high_high,8.0,8.0,16.0,sub500,4607.599515,H19V07
5759926,2.03,5.03,0.505,1.63,1.31,med_high,6.0,8.0,14.0,sub500,4039.230583,H19V07
5759950,6.36,11.30,0.524,6.64,2.75,med_med,5.0,3.0,8.0,sub500,4551.342233,H19V07
5759973,5.75,12.20,0.530,6.44,3.28,high_med,7.0,5.0,12.0,sub500,5154.250000,H19V07


In [14]:
# Create H19V08 dataframe

H19V08_Mask = np.zeros((2400, 2400))
# low_low / low_med / low_high

H19V08_Mask[((H19V08_First_BA<1) * (H19V08_Second_BA<1))==1] = 1
H19V08_Mask[((H19V08_First_BA<1) * (H19V08_Second_BA>=1) * (H19V08_Second_BA<7))==1] = 2
H19V08_Mask[((H19V08_First_BA<1) * (H19V08_Second_BA>=7))==1] = 3

# med_low / med_med / med_high
H19V08_Mask[((H19V08_First_BA>=1) * (H19V08_First_BA<7) * (H19V08_Second_BA<1))==1] = 4
H19V08_Mask[((H19V08_First_BA>=1) * (H19V08_First_BA<7) * (H19V08_Second_BA>=1) * (H19V08_Second_BA<7))==1] = 5
H19V08_Mask[((H19V08_First_BA>=1) * (H19V08_First_BA<7) * (H19V08_Second_BA>=7))==1] = 6

#high_low / high_med / high_high
H19V08_Mask[((H19V08_Second_BA<1) * (H19V08_First_BA>=7))==1] = 7
H19V08_Mask[((H19V08_First_BA>=7) * (H19V08_Second_BA>=1) * (H19V08_Second_BA<7))==1] = 8
H19V08_Mask[((H19V08_First_BA>=7) * (H19V08_Second_BA>=7))==1] = 9

# this takes out where canopy height == 0, but means that the colormaps are hardly visible
# H19V08_Mask[(canopy_height_array==0)==1]=0

H19V08_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_BA_500-1500_H19V08.csv", header=None)
H19V08_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_500-1500_H19V08.csv", header=None)
H19V08_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_BA_sub500_H19V08.csv", header=None)
H19V08_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_sub500_H19V08.csv", header=None)
H19V08_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_above_1500_H19V08.csv", header=None)
H19V08_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_above_1500_H19V08.csv", header=None)
Yearly_H19V08_Rainfall = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H19V08_Rainfall.csv", header=None)

# total BA for each rainfall group
H19V08_sub500_total_mask = H19V08_First_BA_sub500 + H19V08_Second_BA_sub500
H19V08_500_1500_total_mask = H19V08_First_BA_500_1500 + H19V08_Second_BA_500_1500
H19V08_above500_total_mask = H19V08_First_BA_above_1500 + H19V08_Second_BA_above_1500

# VI
H19V08_data = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H19V08_VI_array.csv", header=None)
H19V08_VI_array_np = H19V08_data.to_numpy()
H19V08_VI_list = H19V08_VI_array_np.flatten().tolist()

H19V08_rain_band_mask = np.zeros((2400, 2400))

H19V08_first_BA = H19V08_First_BA_500_1500 + H19V08_First_BA_sub500 + H19V08_First_BA_above_1500
H19V08_second_BA = H19V08_Second_BA_500_1500 + H19V08_Second_BA_sub500 + H19V08_Second_BA_above_1500

for i in range(len(H19V08_sub500_total_mask)):
    for ii in range(len(H19V08_sub500_total_mask)):
        if (H19V08_500_1500_total_mask[i][ii] >= H19V08_sub500_total_mask[i][ii]) & (H19V08_500_1500_total_mask[i][ii] >= H19V08_above500_total_mask[i][ii]):
            H19V08_rain_band_mask[i][ii] = 2
            
        elif (H19V08_above500_total_mask[i][ii] >= H19V08_500_1500_total_mask[i][ii]) & (H19V08_above500_total_mask[i][ii] >= H19V08_sub500_total_mask[i][ii]):
            H19V08_rain_band_mask[i][ii] = 3
            
        elif (H19V08_sub500_total_mask[i][ii] >= H19V08_500_1500_total_mask[i][ii]) & (H19V08_sub500_total_mask[i][ii] >= H19V08_above500_total_mask[i][ii]):
            H19V08_rain_band_mask[i][ii] = 1

H19V08_canopy_height_list = canopy_height_array.flatten().tolist()
H19V08_canopy_max_height_list = canopy_max_height_array.flatten().tolist()
H19V08_canopy_min_height_list = canopy_min_height_array.flatten().tolist()
H19V08_canopy_median_height_list = canopy_median_height_array.flatten().tolist()
H19V08_canopy_openness_list = canopy_openness_array.flatten().tolist()
H19V08_BA_list = H19V08_Mask.flatten().tolist()
H19V08_rain_band_list = H19V08_rain_band_mask.flatten().tolist() 
H19V08_first_BA = H19V08_first_BA.to_numpy()
H19V08_first_BA_list = H19V08_first_BA.flatten().tolist() 
H19V08_second_BA = H19V08_second_BA.to_numpy()
H19V08_second_BA_list = H19V08_second_BA.flatten().tolist() 
Yearly_H19V08_Rainfall = Yearly_H19V08_Rainfall.to_numpy()
Yearly_H19V08_Rainfall_list = Yearly_H19V08_Rainfall.flatten().tolist()
H19V08_total_burns = H19V08_first_BA + H19V08_second_BA
H19V08_total_burns_list = H19V08_total_burns.flatten().tolist() 

H19V08_rain_band_list = map(str, H19V08_rain_band_list)
H19V08_rain_band_list = [i.replace('1.0', 'sub500') for i in H19V08_rain_band_list]
H19V08_rain_band_list = [i.replace('2.0', '500_1500') for i in H19V08_rain_band_list]
H19V08_rain_band_list = [i.replace('3.0', 'above1500') for i in H19V08_rain_band_list]

H19V08_BA_list = map(str, H19V08_BA_list)
H19V08_BA_list = [i.replace('1.0', 'low_low') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('2.0', 'low_med') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('3.0', 'low_high') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('4.0', 'med_low') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('5.0', 'med_med') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('6.0', 'med_high') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('7.0', 'high_low') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('8.0', 'high_med') for i in H19V08_BA_list]
H19V08_BA_list = [i.replace('9.0', 'high_high') for i in H19V08_BA_list]

H19V08_all_data = pd.DataFrame({'mean_canopy': canopy_height_list})

H19V08_all_data['canopy_max_height']= canopy_max_height_list
H19V08_all_data['canopy_min_height']= canopy_min_height_list
H19V08_all_data['canopy_median_height']= canopy_median_height_list
H19V08_all_data['canopy_openness']= canopy_openness_list
H19V08_all_data['rainfall_band']= H19V08_rain_band_list
H19V08_all_data['BA_category']= H19V08_BA_list
H19V08_all_data['Total rainfall']= Yearly_H19V08_Rainfall_list
H19V08_all_data['First_burns']= H19V08_first_BA_list
H19V08_all_data['Second_burns']= H19V08_second_BA_list
H19V08_all_data['Total burns'] = H19V08_total_burns_list
H19V08_all_data['Average NDVI']= H19V08_VI_list

H19V08_all_data =H19V08_all_data[H19V08_all_data['mean_canopy'] !=0]

H19V08_all_data['Tile']= "H19V08"

In [15]:
# Create H20V07 dataframe

H20V07_Mask = np.zeros((2400, 2400))
# low_low / low_med / low_high

H20V07_Mask[((H20V07_First_BA<1) * (H20V07_Second_BA<1))==1] = 1
H20V07_Mask[((H20V07_First_BA<1) * (H20V07_Second_BA>=1) * (H20V07_Second_BA<7))==1] = 2
H20V07_Mask[((H20V07_First_BA<1) * (H20V07_Second_BA>=7))==1] = 3

# med_low / med_med / med_high
H20V07_Mask[((H20V07_First_BA>=1) * (H20V07_First_BA<7) * (H20V07_Second_BA<1))==1] = 4
H20V07_Mask[((H20V07_First_BA>=1) * (H20V07_First_BA<7) * (H20V07_Second_BA>=1) * (H20V07_Second_BA<7))==1] = 5
H20V07_Mask[((H20V07_First_BA>=1) * (H20V07_First_BA<7) * (H20V07_Second_BA>=7))==1] = 6

#high_low / high_med / high_high
H20V07_Mask[((H20V07_Second_BA<1) * (H20V07_First_BA>=7))==1] = 7
H20V07_Mask[((H20V07_First_BA>=7) * (H20V07_Second_BA>=1) * (H20V07_Second_BA<7))==1] = 8
H20V07_Mask[((H20V07_First_BA>=7) * (H20V07_Second_BA>=7))==1] = 9

# this takes out where canopy height == 0, but means that the colormaps are hardly visible
# H20V07_Mask[(canopy_height_array==0)==1]=0

H20V07_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\first_BA_500-1500_H20V07.csv", header=None)
H20V07_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\second_BA_500-1500_H20V07.csv", header=None)
H20V07_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\first_BA_sub500_H20V07.csv", header=None)
H20V07_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\second_BA_sub500_H20V07.csv", header=None)
H20V07_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\first_above_1500_H20V07.csv", header=None)
H20V07_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\second_BA_above_1500_H20V07.csv", header=None)
Yearly_H20V07_Rainfall = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H20V07_Rainfall.csv", header=None)

# total BA for each rainfall group
H20V07_sub500_total_mask = H20V07_First_BA_sub500 + H20V07_Second_BA_sub500
H20V07_500_1500_total_mask = H20V07_First_BA_500_1500 + H20V07_Second_BA_500_1500
H20V07_above500_total_mask = H20V07_First_BA_above_1500 + H20V07_Second_BA_above_1500

# VI
H20V07_data = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H20V07_VI_array.csv", header=None)
H20V07_VI_array_np = H20V07_data.to_numpy()
H20V07_VI_list = H20V07_VI_array_np.flatten().tolist()

H20V07_first_BA = H20V07_First_BA_500_1500 + H20V07_First_BA_sub500 + H20V07_First_BA_above_1500
H20V07_second_BA = H20V07_Second_BA_500_1500 + H20V07_Second_BA_sub500 + H20V07_Second_BA_above_1500

H20V07_rain_band_mask = np.zeros((2400, 2400))

for i in range(len(H20V07_sub500_total_mask)):
    for ii in range(len(H20V07_sub500_total_mask)):
        if (H20V07_500_1500_total_mask[i][ii] >= H20V07_sub500_total_mask[i][ii]) & (H20V07_500_1500_total_mask[i][ii] >= H20V07_above500_total_mask[i][ii]):
            H20V07_rain_band_mask[i][ii] = 2
            
        elif (H20V07_above500_total_mask[i][ii] >= H20V07_500_1500_total_mask[i][ii]) & (H20V07_above500_total_mask[i][ii] >= H20V07_sub500_total_mask[i][ii]):
            H20V07_rain_band_mask[i][ii] = 3
            
        elif (H20V07_sub500_total_mask[i][ii] >= H20V07_500_1500_total_mask[i][ii]) & (H20V07_sub500_total_mask[i][ii] >= H20V07_above500_total_mask[i][ii]):
            H20V07_rain_band_mask[i][ii] = 1

H20V07_canopy_height_list = canopy_height_array.flatten().tolist()
H20V07_canopy_max_height_list = canopy_max_height_array.flatten().tolist()
H20V07_canopy_min_height_list = canopy_min_height_array.flatten().tolist()
H20V07_canopy_median_height_list = canopy_median_height_array.flatten().tolist()
H20V07_canopy_openness_list = canopy_openness_array.flatten().tolist()
H20V07_BA_list = H20V07_Mask.flatten().tolist()
H20V07_rain_band_list = H20V07_rain_band_mask.flatten().tolist()
H20V07_first_BA = H20V07_first_BA.to_numpy()
H20V07_first_BA_list = H20V07_first_BA.flatten().tolist() 
H20V07_second_BA = H20V07_second_BA.to_numpy()
H20V07_second_BA_list = H20V07_second_BA.flatten().tolist() 
Yearly_H20V07_Rainfall = Yearly_H20V07_Rainfall.to_numpy()
Yearly_H20V07_Rainfall_list = Yearly_H20V07_Rainfall.flatten().tolist()
H20V07_total_burns = H20V07_first_BA + H20V07_second_BA
H20V07_total_burns_list = H20V07_total_burns.flatten().tolist() 

H20V07_rain_band_list = map(str, H20V07_rain_band_list)
H20V07_rain_band_list = [i.replace('1.0', 'sub500') for i in H20V07_rain_band_list]
H20V07_rain_band_list = [i.replace('2.0', '500_1500') for i in H20V07_rain_band_list]
H20V07_rain_band_list = [i.replace('3.0', 'above1500') for i in H20V07_rain_band_list]

H20V07_BA_list = map(str, H20V07_BA_list)
H20V07_BA_list = [i.replace('1.0', 'low_low') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('2.0', 'low_med') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('3.0', 'low_high') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('4.0', 'med_low') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('5.0', 'med_med') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('6.0', 'med_high') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('7.0', 'high_low') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('8.0', 'high_med') for i in H20V07_BA_list]
H20V07_BA_list = [i.replace('9.0', 'high_high') for i in H20V07_BA_list]

H20V07_all_data = pd.DataFrame({'mean_canopy': canopy_height_list})

H20V07_all_data['canopy_max_height']= canopy_max_height_list
H20V07_all_data['canopy_min_height']= canopy_min_height_list
H20V07_all_data['canopy_median_height']= canopy_median_height_list
H20V07_all_data['canopy_openness']= canopy_openness_list
H20V07_all_data['rainfall_band']= H20V07_rain_band_list
H20V07_all_data['BA_category']= H20V07_BA_list
H20V07_all_data['Total rainfall']= Yearly_H20V07_Rainfall_list
H20V07_all_data['First_burns']= H20V07_first_BA_list
H20V07_all_data['Second_burns']= H20V07_second_BA_list
H20V07_all_data['Total burns'] = H20V07_total_burns_list
H20V07_all_data['Average NDVI']= H20V07_VI_list

H20V07_all_data =H20V07_all_data[H20V07_all_data['mean_canopy'] !=0]

H20V07_all_data['Tile']= "H20V07"

In [16]:
# Create H20V08 dataframe

H20V08_Mask = np.zeros((2400, 2400))
# low_low / low_med / low_high

H20V08_Mask[((H20V08_First_BA<1) * (H20V08_Second_BA<1))==1] = 1
H20V08_Mask[((H20V08_First_BA<1) * (H20V08_Second_BA>=1) * (H20V08_Second_BA<7))==1] = 2
H20V08_Mask[((H20V08_First_BA<1) * (H20V08_Second_BA>=7))==1] = 3

# med_low / med_med / med_high
H20V08_Mask[((H20V08_First_BA>=1) * (H20V08_First_BA<7) * (H20V08_Second_BA<1))==1] = 4
H20V08_Mask[((H20V08_First_BA>=1) * (H20V08_First_BA<7) * (H20V08_Second_BA>=1) * (H20V08_Second_BA<7))==1] = 5
H20V08_Mask[((H20V08_First_BA>=1) * (H20V08_First_BA<7) * (H20V08_Second_BA>=7))==1] = 6

#high_low / high_med / high_high
H20V08_Mask[((H20V08_Second_BA<1) * (H20V08_First_BA>=7))==1] = 7
H20V08_Mask[((H20V08_First_BA>=7) * (H20V08_Second_BA>=1) * (H20V08_Second_BA<7))==1] = 8
H20V08_Mask[((H20V08_First_BA>=7) * (H20V08_Second_BA>=7))==1] = 9

# this takes out where canopy height == 0, but means that the colormaps are hardly visible
# H20V08_Mask[(canopy_height_array==0)==1]=0

H20V08_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_500-1500_H20V08.csv", header=None)
H20V08_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_500-1500_H20V08.csv", header=None)
H20V08_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_sub500_H20V08.csv", header=None)
H20V08_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_sub500_H20V08.csv", header=None)
H20V08_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_above1500_H20V08.csv", header=None)
H20V08_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_above1500_H20V08.csv", header=None)
Yearly_H20V08_Rainfall = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H20V08_Rainfall.csv", header=None)

# total BA for each rainfall group
H20V08_sub500_total_mask = H20V08_First_BA_sub500 + H20V08_Second_BA_sub500
H20V08_500_1500_total_mask = H20V08_First_BA_500_1500 + H20V08_Second_BA_500_1500
H20V08_above500_total_mask = H20V08_First_BA_above_1500 + H20V08_Second_BA_above_1500

# VI
H20V08_data = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H20V08_VI_array.csv", header=None)
H20V08_VI_array_np = H20V08_data.to_numpy()
H20V08_VI_list = H20V08_VI_array_np.flatten().tolist()

H20V08_first_BA = H20V08_First_BA_500_1500 + H20V08_First_BA_sub500 + H20V08_First_BA_above_1500
H20V08_second_BA = H20V08_Second_BA_500_1500 + H20V08_Second_BA_sub500 + H20V08_Second_BA_above_1500

H20V08_rain_band_mask = np.zeros((2400, 2400))

for i in range(len(H20V08_sub500_total_mask)):
    for ii in range(len(H20V08_sub500_total_mask)):
        if (H20V08_500_1500_total_mask[i][ii] >= H20V08_sub500_total_mask[i][ii]) & (H20V08_500_1500_total_mask[i][ii] >= H20V08_above500_total_mask[i][ii]):
            H20V08_rain_band_mask[i][ii] = 2
            
        elif (H20V08_above500_total_mask[i][ii] >= H20V08_500_1500_total_mask[i][ii]) & (H20V08_above500_total_mask[i][ii] >= H20V08_sub500_total_mask[i][ii]):
            H20V08_rain_band_mask[i][ii] = 3
            
        elif (H20V08_sub500_total_mask[i][ii] >= H20V08_500_1500_total_mask[i][ii]) & (H20V08_sub500_total_mask[i][ii] >= H20V08_above500_total_mask[i][ii]):
            H20V08_rain_band_mask[i][ii] = 1

H20V08_canopy_height_list = canopy_height_array.flatten().tolist()
H20V08_canopy_max_height_list = canopy_max_height_array.flatten().tolist()
H20V08_canopy_min_height_list = canopy_min_height_array.flatten().tolist()
H20V08_canopy_median_height_list = canopy_median_height_array.flatten().tolist()
H20V08_canopy_openness_list = canopy_openness_array.flatten().tolist()
H20V08_BA_list = H20V08_Mask.flatten().tolist()
H20V08_rain_band_list = H20V08_rain_band_mask.flatten().tolist()
H20V08_first_BA = H20V08_first_BA.to_numpy()
H20V08_first_BA_list = H20V08_first_BA.flatten().tolist() 
H20V08_second_BA = H20V08_second_BA.to_numpy()
H20V08_second_BA_list = H20V08_second_BA.flatten().tolist() 
Yearly_H20V08_Rainfall = Yearly_H20V08_Rainfall.to_numpy()
Yearly_H20V08_Rainfall_list = Yearly_H20V08_Rainfall.flatten().tolist()
H20V08_total_burns = H20V08_first_BA + H20V08_second_BA
H20V08_total_burns_list = H20V08_total_burns.flatten().tolist() 

H20V08_rain_band_list = map(str, H20V08_rain_band_list)
H20V08_rain_band_list = [i.replace('1.0', 'sub500') for i in H20V08_rain_band_list]
H20V08_rain_band_list = [i.replace('2.0', '500_1500') for i in H20V08_rain_band_list]
H20V08_rain_band_list = [i.replace('3.0', 'above1500') for i in H20V08_rain_band_list]

H20V08_BA_list = map(str, H20V08_BA_list)
H20V08_BA_list = [i.replace('1.0', 'low_low') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('2.0', 'low_med') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('3.0', 'low_high') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('4.0', 'med_low') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('5.0', 'med_med') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('6.0', 'med_high') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('7.0', 'high_low') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('8.0', 'high_med') for i in H20V08_BA_list]
H20V08_BA_list = [i.replace('9.0', 'high_high') for i in H20V08_BA_list]

H20V08_all_data = pd.DataFrame({'mean_canopy': canopy_height_list})

H20V08_all_data['canopy_max_height']= canopy_max_height_list
H20V08_all_data['canopy_min_height']= canopy_min_height_list
H20V08_all_data['canopy_median_height']= canopy_median_height_list
H20V08_all_data['canopy_openness']= canopy_openness_list
H20V08_all_data['rainfall_band']= H20V08_rain_band_list
H20V08_all_data['BA_category']= H20V08_BA_list
H20V08_all_data['Total rainfall']= Yearly_H20V08_Rainfall_list
H20V08_all_data['First_burns']= H20V08_first_BA_list
H20V08_all_data['Second_burns']= H20V08_second_BA_list
H20V08_all_data['Total burns'] = H20V08_total_burns_list
H20V08_all_data['Average NDVI']= H20V08_VI_list

H20V08_all_data =H20V08_all_data[H20V08_all_data['mean_canopy'] !=0]

H20V08_all_data['Tile']= "H20V08"

In [47]:
total_data = H19V07_all_data.append(H19V08_all_data, ignore_index=True)
total_data = total_data.append(H20V07_all_data, ignore_index=True)
total_data = total_data.append(H20V08_all_data, ignore_index=True)

In [48]:
print(len(total_data))
print(len(H19V07_all_data) * 4)
print(len(H19V07_all_data))
print(len(H19V08_all_data))
print(len(H20V07_all_data))
print(len(H20V08_all_data))

305308
305308
76327
76327
76327
76327


In [49]:
data_H19V07 = total_data.loc[total_data['Tile'] == "H19V07"]
data_H19V08 = total_data.loc[total_data['Tile'] == "H19V08"]
data_H20V07 = total_data.loc[total_data['Tile'] == "H20V07"]
data_H20V08 = total_data.loc[total_data['Tile'] == "H20V08"]

In [61]:
print(data_H19V07['Total rainfall'].min())
print(data_H19V08['Total rainfall'].min())
print(data_H20V07['Total rainfall'].min())
print(data_H20V08['Total rainfall'].min())
print()
print(data_H19V07['Total rainfall'].max())
print(data_H19V08['Total rainfall'].max())
print(data_H20V07['Total rainfall'].max())
print(data_H20V08['Total rainfall'].max())
print()
print(data_H19V07['Total rainfall'].mean())
print(data_H19V08['Total rainfall'].mean())
print(data_H20V07['Total rainfall'].mean())
print(data_H20V08['Total rainfall'].mean())

8.101823656685726
821.692936490417
729.4401820117649
729.4401820117649

1085.8308986612249
3231.405057316646
2317.2141814601296
2317.2141814601296

309.0052192628396
1524.4379356294498
1533.9732360998412
1533.9732360998412


In [51]:
np.savetxt("D:\\masters_data\\output\\csv\\Analysis\\total_data.csv", total_data, delimiter=",", fmt='%s')

In [52]:
H19V07 = total_data.loc[total_data['Tile'] == "H19V07"]

In [55]:
H19V07

,mean_canopy,canopy_max_height,canopy_min_height,canopy_median_height,canopy_openness,BA_category,Total rainfall,First_burns,Second_burns,Total burns,rainfall_band,Average NDVI,Tile
0,6.04,9.64,2.290,6.42,1.97,low_low,26.073093,0.0,0.0,0.0,500_1500,979.815534,H19V07
1,2.26,5.38,0.511,1.74,1.40,low_low,26.068037,0.0,0.0,0.0,500_1500,986.992718,H19V07
2,2.53,7.31,0.514,1.18,2.25,low_low,26.160865,0.0,0.0,0.0,500_1500,972.259709,H19V07
3,5.72,8.51,0.622,5.94,2.13,low_low,29.026940,0.0,0.0,0.0,500_1500,976.728155,H19V07
4,5.20,8.15,0.520,5.08,1.86,low_low,27.917566,0.0,0.0,0.0,500_1500,978.490291,H19V07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76322,2.32,7.27,0.537,1.46,1.89,high_high,863.343368,8.0,8.0,16.0,sub500,4607.599515,H19V07
76323,2.03,5.03,0.505,1.63,1.31,med_high,878.386377,6.0,8.0,14.0,sub500,4039.230583,H19V07
76324,6.36,11.30,0.524,6.64,2.75,med_med,873.237738,5.0,3.0,8.0,sub500,4551.342233,H19V07
76325,5.75,12.20,0.530,6.44,3.28,high_med,886.250499,7.0,5.0,12.0,sub500,5154.250000,H19V07


In [56]:
total_data

,mean_canopy,canopy_max_height,canopy_min_height,canopy_median_height,canopy_openness,BA_category,Total rainfall,First_burns,Second_burns,Total burns,rainfall_band,Average NDVI,Tile
0,6.04,9.64,2.290,6.42,1.97,low_low,26.073093,0.0,0.0,0.0,500_1500,979.815534,H19V07
1,2.26,5.38,0.511,1.74,1.40,low_low,26.068037,0.0,0.0,0.0,500_1500,986.992718,H19V07
2,2.53,7.31,0.514,1.18,2.25,low_low,26.160865,0.0,0.0,0.0,500_1500,972.259709,H19V07
3,5.72,8.51,0.622,5.94,2.13,low_low,29.026940,0.0,0.0,0.0,500_1500,976.728155,H19V07
4,5.20,8.15,0.520,5.08,1.86,low_low,27.917566,0.0,0.0,0.0,500_1500,978.490291,H19V07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
305303,2.32,7.27,0.537,1.46,1.89,med_high,1721.141523,6.0,8.0,14.0,500_1500,5547.019417,H20V08
305304,2.03,5.03,0.505,1.63,1.31,med_med,1436.409851,6.0,3.0,9.0,500_1500,5708.546117,H20V08
305305,6.36,11.30,0.524,6.64,2.75,low_low,1196.989934,0.0,0.0,0.0,500_1500,5775.082524,H20V08
305306,5.75,12.20,0.530,6.44,3.28,low_low,1042.818575,0.0,0.0,0.0,500_1500,5613.589806,H20V08
